# **Import and Preparation**
---
Import the required library, set global variables and create generator and discriminator

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!nvidia-smi

Sun Aug 23 01:46:23 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Import Library

In [3]:
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 143kB 2.7MB/s 


In [4]:
import easydict
import os
import argparse
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn, optim, autograd

import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset

import torchvision
from torchvision.utils import save_image
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder, DatasetFolder

## Global Variables

In [139]:
opt = easydict.EasyDict({
    "n_epochs": 200,
    "batch_size":128,
    "lr": 0.0002,
    "b1": 0.5,
    "b2": 0.999,
    "n_cpu": 0,
    "latent_dim": 100,
    "n_classes": 10,
    "img_size": 32,
    "channels": 3,
    "sample_size": 2000,
    "flipped_prop": 0.05,
    "name": "SVHN",
    
})
print(opt)


img_shape = (opt.channels, opt.img_size, opt.img_size)

cuda = True if torch.cuda.is_available() else False

{'n_epochs': 200, 'batch_size': 128, 'lr': 0.0002, 'b1': 0.5, 'b2': 0.999, 'n_cpu': 0, 'latent_dim': 100, 'n_classes': 10, 'img_size': 32, 'channels': 3, 'sample_size': 2000, 'flipped_prop': 0.05, 'name': 'SVHN'}


## Generator

In [108]:
class Generator(nn.Module):
    # initializers
    def __init__(self, d=128):
        super(Generator, self).__init__()
        self.deconv1_1 = nn.ConvTranspose2d(100, d*2, 4, 1, 0)
        self.deconv1_1_bn = nn.BatchNorm2d(d*2) # 4
        self.deconv1_2 = nn.ConvTranspose2d(10, d*2, 4, 1, 0)
        self.deconv1_2_bn = nn.BatchNorm2d(d*2) # 4
        self.deconv2 = nn.ConvTranspose2d(d*4, d*2, 4, 2, 1)
        self.deconv2_bn = nn.BatchNorm2d(d*2) # 8
        self.deconv3 = nn.ConvTranspose2d(d*2, d, 4, 2, 1)
        self.deconv3_bn = nn.BatchNorm2d(d) # 16
        self.deconv4 = nn.ConvTranspose2d(d,  opt.channels, 4, 2, 1) 

    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    # forward method
    def forward(self, noise, labels):
        x = F.relu(self.deconv1_1_bn(self.deconv1_1(noise)))
        y = F.relu(self.deconv1_2_bn(self.deconv1_2(labels)))
        x = torch.cat([x, y], 1)
        x = F.relu(self.deconv2_bn(self.deconv2(x)))
        x = F.relu(self.deconv3_bn(self.deconv3(x)))
        # x = F.relu(self.deconv4_bn(self.deconv4(x)))
        x = F.tanh(self.deconv4(x))
        # x = F.relu(self.deconv5_bn(self.deconv5(x)))
        # x = F.tanh(self.deconv6(x))
        # x = F.relu(self.deconv4_bn(self.deconv4(x)))
        # x = F.tanh(self.deconv5(x))

        return x

## Discriminator
Discriminator_SGAN is the Discriminator of SGAN which has N+1 outputs. The proposed CDCGAN's Discriminator has 2N outputs 

In [109]:
class Discriminator(nn.Module):
    # initializers
    def __init__(self, d=128):
        super(Discriminator, self).__init__()
        self.conv1_1 = nn.Conv2d(opt.channels, int(d/2), 4, 2, 1) # 64
        self.conv1_2 = nn.Conv2d(10, int(d/2), 4, 2, 1) # 64
        self.conv2 = nn.Conv2d(d, d*2, 4, 2, 1) 
        self.conv2_bn = nn.BatchNorm2d(d*2) # 32
        self.conv3 = nn.Conv2d(d*2, d*4, 4, 2, 1)
        self.conv3_bn = nn.BatchNorm2d(d*4) # 16
        self.conv4 = nn.Conv2d(d * 4, 20, 4, 1, 0)

        # self.conv4_bn = nn.BatchNorm2d(d*8) # 8
        # self.conv5 = nn.Conv2d(d * 8, 20, 4, 1, 0)
        # self.conv5_bn = nn.BatchNorm2d(d*16) # 4
        # self.conv6 = nn.Conv2d(d * 16, 20, 4, 1, 0)

    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    # forward method
    def forward(self, imgs, labels):
        x = F.leaky_relu(self.conv1_1(imgs), 0.2)
        y = F.leaky_relu(self.conv1_2(labels), 0.2)
        x = torch.cat([x, y], 1)
        x = F.leaky_relu(self.conv2_bn(self.conv2(x)), 0.2)
        hidden = x
        x = F.leaky_relu(self.conv3_bn(self.conv3(x)), 0.2)
        # x = F.leaky_relu(self.conv4_bn(self.conv4(x)), 0.2)
        x = F.sigmoid(self.conv4(x))
        # x = F.leaky_relu(self.conv5_bn(self.conv5(x)), 0.2)
        # x = F.sigmoid(self.conv6(x))

        return x, hidden


In [110]:
class Discriminator_SGAN(nn.Module):
    # initializers
    def __init__(self, d=128):
        super(Discriminator_SGAN, self).__init__()
        self.conv1_1 = nn.Conv2d(opt.channels, int(d/2), 4, 2, 1) # 64
        self.conv1_2 = nn.Conv2d(10, int(d/2), 4, 2, 1) # 64
        self.conv2 = nn.Conv2d(d, d*2, 4, 2, 1) 
        self.conv2_bn = nn.BatchNorm2d(d*2) # 32
        self.conv3 = nn.Conv2d(d*2, d*4, 4, 2, 1)
        self.conv3_bn = nn.BatchNorm2d(d*4) # 16
        self.conv4 = nn.Conv2d(d * 4, 11, 4, 1, 0)

        # self.conv4_bn = nn.BatchNorm2d(d*8) # 8
        # self.conv5 = nn.Conv2d(d * 8, 20, 4, 1, 0)
        # self.conv5_bn = nn.BatchNorm2d(d*16) # 4
        # self.conv6 = nn.Conv2d(d * 16, 20, 4, 1, 0)

    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    # forward method
    def forward(self, imgs, labels):
        x = F.leaky_relu(self.conv1_1(imgs), 0.2)
        y = F.leaky_relu(self.conv1_2(labels), 0.2)
        x = torch.cat([x, y], 1)
        x = F.leaky_relu(self.conv2_bn(self.conv2(x)), 0.2)
        hidden = x
        x = F.leaky_relu(self.conv3_bn(self.conv3(x)), 0.2)
        # x = F.leaky_relu(self.conv4_bn(self.conv4(x)), 0.2)
        x = F.sigmoid(self.conv4(x))
        # x = F.leaky_relu(self.conv5_bn(self.conv5(x)), 0.2)
        # x = F.sigmoid(self.conv6(x))

        return x, hidden


In [10]:
# Initial Weight

def normal_init(m, mean, std):
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
        m.weight.data.normal_(mean, std)
        m.bias.data.zero_()

# **Create Dataset for Few-Shot Learning**

---

Take different Numbers of training samples (50,100,200,500,1000,2000) from the downloaded dataset.

## Dataset

In [11]:
class MyDataset(Dataset):

    def __init__(self, data_tensor, target_tensor, transform):
        self.data_tensor = data_tensor
        self.target_tensor = target_tensor
        self.transform = transform

    def __getitem__(self, index):
        return (self.data_tensor[index], self.target_tensor[index])

    def __len__(self):
        return len(self.data_tensor)

## Load Data

In [12]:
 def load_data(Name):
    if Name == 'MNIST':
      # Configure data loader
      os.makedirs("/content/drive/My Drive/Summer Project/MNIST", exist_ok=True)
      dataset = datasets.MNIST(
        "/content/drive/My Drive/Summer Project/MNIST",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
      )
    elif Name == "FashionMNIST":
      # Configure data loader
      os.makedirs("/content/drive/My Drive/Summer Project/FashionMNIST", exist_ok=True)
      dataset = datasets.FashionMNIST(
        "/content/drive/My Drive/Summer Project/FashionMNIST",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
      )
    elif Name == "SVHN":
    # Configure data loader
      os.makedirs("/content/drive/My Drive/Summer Project/SVHN", exist_ok=True)
      dataset = datasets.SVHN(
        "/content/drive/My Drive/Summer Project/SVHN",
        split = "train",
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
      )
    elif Name == "CIFAR10":
      os.makedirs("/content/drive/My Drive/Summer Project/CIFAR10", exist_ok=True)
      dataset = datasets.CIFAR10(
          "/content/drive/My Drive/Summer Project/CIFAR10",
          train=True,
          download=True,
          transform=transforms.Compose(
              [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
          ),
      )
    
    dataloader = torch.utils.data.DataLoader(
      dataset,
      batch_size=1,
      shuffle=False,
    )

    index_labels = []
    images = [] 
    for j in range(opt.n_classes):
        y = []
        x = []
        for i, (img, label) in enumerate(dataloader):
            if label == j:
                y.append(i)
                img = img.reshape(opt.channels,opt.img_size,opt.img_size)
                x.append(img)
        index_labels.append(y)
        images.append(x)
        
    train_data = []
    train_labels = []

    for i in range(opt.n_classes):
      for j in range(opt.sample_size):
          num = np.random.randint(0,len(images[i]))
          train_data.append(images[i][num])
          train_labels.append(i)

    mydataset = MyDataset(train_data, train_labels, transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ))
    
    loader = torch.utils.data.DataLoader(
        mydataset,
        batch_size=opt.batch_size,
        shuffle=True,
    )
    return loader


def load_test(Name):
  if Name == "MNIST":
    # Configure data loader
    os.makedirs("/content/drive/My Drive/Summer Project/MNIST", exist_ok=True)
    dataset = datasets.MNIST(
            "/content/drive/My Drive/Summer Project/MNIST",
            train=False,
            download=True,
            transform=transforms.Compose(
                [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
            ),
        )
  elif Name == "FashionMNIST":
    os.makedirs("/content/drive/My Drive/Summer Project/FashionMNIST", exist_ok=True)
    dataset = datasets.FashionMNIST(
          "/content/drive/My Drive/Summer Project/FashionMNIST",
          train=False,
          download=True,
          transform=transforms.Compose(
              [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
          ),
      )
  elif Name == "SVHN":
    os.makedirs("/content/drive/My Drive/Summer Project/SVHN", exist_ok=True)
    dataset = datasets.SVHN(
          "/content/drive/My Drive/Summer Project/SVHN",
          split = "test",
          download=True,
          transform=transforms.Compose(
              [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
          ),
      )
  elif Name == "CIFAR10":
    os.makedirs("/content/drive/My Drive/Summer Project/CIFAR10", exist_ok=True)
    dataset = datasets.CIFAR10(
          "/content/drive/My Drive/Summer Project/CIFAR10",
          train=False,
          download=True,
          transform=transforms.Compose(
              [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
          ),
      )
    
  testloader = torch.utils.data.DataLoader(
      dataset,
      batch_size=64,
      shuffle=False,
  )

  return testloader

In [140]:
# Load Data

dataloader = load_data(opt.name)
testloader = load_test(opt.name)

Using downloaded and verified file: /content/drive/My Drive/Summer Project/SVHN/train_32x32.mat
Using downloaded and verified file: /content/drive/My Drive/Summer Project/SVHN/test_32x32.mat


## Initial Generator and Discriminator

In [104]:
# # Loss functions
adversarial_loss = torch.nn.BCELoss()
classification_loss = torch.nn.CrossEntropyLoss()
classification_loss2 = torch.nn.MSELoss()
# # Initialize generator and discriminator
generator = Generator(128)
discriminator = Discriminator(128)
generator_S = Generator(128)
discriminator_S = Discriminator_SGAN(128)

generator.weight_init(mean=0.0, std=0.02)
discriminator.weight_init(mean=0.0, std=0.02)
generator_S.weight_init(mean=0.0, std=0.02)
discriminator_S.weight_init(mean=0.0, std=0.02)
if cuda:
    generator.cuda()
    discriminator.cuda()
    generator_S.cuda()
    discriminator_S.cuda()
    adversarial_loss.cuda()
    classification_loss.cuda()
    classification_loss2.cuda()

# # Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0005, betas=(opt.b1, opt.b2))
optimizer_SG = torch.optim.Adam(generator_S.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_SD = torch.optim.Adam(discriminator_S.parameters(), lr=0.0005, betas=(opt.b1, opt.b2))
optimizer_A = torch.optim.Adam([{"params":generator.parameters()},{"params":discriminator.parameters()}], lr=0.01, betas=(opt.b1, opt.b2))

FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

In [ ]:
print(discriminator)

In [ ]:
print(generator)

## Train Function 

### Proposed Model

In [16]:
# mean = torch.Tensor(opt.n_classes, 1).uniform_(0.0, 1.0)
# std = torch.rand(opt.n_classes, 1)

In [17]:
def generator_train(batch_size, valid, imgs, labels):
    optimizer_G.zero_grad()

    noise = torch.randn((batch_size, 100)).view(-1, 100, 1, 1)
    # noise = mean[labels].reshape(8,1) + noise * std[labels].reshape(8,1)
    # noise = noise.view(-1, 100, 1, 1)

    # noise = torch.normal(mean, std)
    # fake_labels = (torch.rand(batch_size, 1) * 10).type(torch.LongTensor).squeeze()
    # fake_labels = (torch.rand(batch_size, 1) * 10).type(torch.LongTensor).reshape(1)
    gen_fake_labels = onehot[labels]
    dis_fake_labels = fill[labels]
    noise, gen_fake_labels, dis_fake_labels = Variable(noise.cuda()), Variable(gen_fake_labels.cuda()), Variable(dis_fake_labels.cuda())

    gen_imgs = generator(noise, gen_fake_labels)

    validity, _ = discriminator(gen_imgs, dis_fake_labels)
    validity = validity.squeeze()
    # regularization = torch.norm(imgs.cpu() - gen_imgs.cpu())
    # regularization = Variable(regularization.cuda())
    loss_G = adversarial_loss(validity, valid)
    # loss_G = adversarial_loss(validity, valid)

    loss_G.backward()
    optimizer_G.step()
    # plt.imshow(gen_imgs[0].cpu().detach().numpy().reshape(32,32,3))
    # plt.show()
    return loss_G

In [18]:
def discriminator_train(batch_size, imgs, labels, valid, fake):
    optimizer_D.zero_grad()
    real_labels = fill[labels]
    imgs, real_labels = Variable(imgs.cuda()), Variable(real_labels.cuda())
    validity_real, hidden1 = discriminator(imgs, real_labels)
    validity_real = validity_real.squeeze()
    d_real_loss = adversarial_loss(validity_real, valid)

    noise = torch.randn((batch_size, 100)).view(-1, 100, 1, 1)
    # fake_labels = (torch.rand(batch_size, 1) * 10).type(torch.LongTensor).squeeze()
    # fake_labels = (torch.rand(batch_size, 1) * 10).type(torch.LongTensor).reshape(1)
    gen_fake_labels = onehot[labels]
    noise, gen_fake_labels = Variable(noise.cuda()), Variable(gen_fake_labels.cuda())
    gen_imgs = generator(noise, gen_fake_labels)
    
    validity_fake, hidden2 = discriminator(gen_imgs, real_labels)
    validity_fake = validity_fake.squeeze()
    d_fake_loss = adversarial_loss(validity_fake, fake)
    # gp = calc_gradient_penalty(discriminator, imgs, gen_imgs, batch_size, labels)
    # regularization = torch.norm((hidden1.mean() - hidden2.mean()).cuda())
    loss_D = d_real_loss + d_fake_loss
    # loss_D = d_real_loss + d_fake_loss 
    loss_D.backward()
    optimizer_D.step()
    return loss_D

### SGAN Model

In [19]:
def generator_S_train(batch_size, valid, imgs, labels):
    optimizer_SG.zero_grad()

    noise = torch.randn((batch_size, 100)).view(-1, 100, 1, 1)
    # noise = mean[labels].reshape(8,1) + noise * std[labels].reshape(8,1)
    # noise = noise.view(-1, 100, 1, 1)

    # noise = torch.normal(mean, std)
    # fake_labels = (torch.rand(batch_size, 1) * 10).type(torch.LongTensor).squeeze()
    # fake_labels = (torch.rand(batch_size, 1) * 10).type(torch.LongTensor).reshape(1)
    gen_fake_labels = onehot[labels]
    dis_fake_labels = fill[labels]
    noise, gen_fake_labels, dis_fake_labels = Variable(noise.cuda()), Variable(gen_fake_labels.cuda()), Variable(dis_fake_labels.cuda())

    gen_imgs = generator_S(noise, gen_fake_labels)

    validity, _ = discriminator_S(gen_imgs, dis_fake_labels)
    validity = validity.squeeze()
    # regularization = torch.norm(imgs.cpu() - gen_imgs.cpu())
    # regularization = Variable(regularization.cuda())
    loss_G = adversarial_loss(validity, valid)
    # loss_G = adversarial_loss(validity, valid)

    loss_G.backward()
    optimizer_SG.step()
    # plt.imshow(gen_imgs[0].cpu().detach().numpy().reshape(32,32,3))
    # plt.show()
    return loss_G

In [20]:
def discriminator_S_train(batch_size, imgs, labels, valid, fake):
    optimizer_SD.zero_grad()
    real_labels = fill[labels]
    imgs, real_labels = Variable(imgs.cuda()), Variable(real_labels.cuda())
    validity_real, hidden1 = discriminator_S(imgs, real_labels)
    validity_real = validity_real.squeeze()
    d_real_loss = adversarial_loss(validity_real, valid)

    noise = torch.randn((batch_size, 100)).view(-1, 100, 1, 1)
    # fake_labels = (torch.rand(batch_size, 1) * 10).type(torch.LongTensor).squeeze()
    # fake_labels = (torch.rand(batch_size, 1) * 10).type(torch.LongTensor).reshape(1)
    gen_fake_labels = onehot[labels]
    noise, gen_fake_labels = Variable(noise.cuda()), Variable(gen_fake_labels.cuda())
    gen_imgs = generator_S(noise, gen_fake_labels)
    
    validity_fake, hidden2 = discriminator_S(gen_imgs, real_labels)
    validity_fake = validity_fake.squeeze()
    d_fake_loss = adversarial_loss(validity_fake, fake)
    # gp = calc_gradient_penalty(discriminator, imgs, gen_imgs, batch_size, labels)
    # regularization = torch.norm((hidden1.mean() - hidden2.mean()).cuda())
    loss_D = d_real_loss + d_fake_loss
    # loss_D = d_real_loss + d_fake_loss 
    loss_D.backward()
    optimizer_SD.step()
    return loss_D

### GP Function

In [21]:
def calc_gradient_penalty(netD, real_data, fake_data, batch_size, labels):
    # print "real_data: ", real_data.size(), fake_data.size()
    alpha = torch.rand(batch_size, 1)

    alpha = alpha.expand(batch_size, int(real_data.nelement()/batch_size)).contiguous().view(batch_size, 3, 32, 32)
    alpha = alpha.cuda() if cuda else alpha

    interpolates = alpha * real_data + ((1 - alpha) * fake_data)

    if cuda:
        interpolates = interpolates.cuda()
    interpolates = autograd.Variable(interpolates, requires_grad=True)
    fill_labels = fill[labels]
    fill_labels = Variable(fill_labels.cuda())
    disc_interpolates, _ = netD(interpolates, fill_labels)

    gradients = autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                              grad_outputs=torch.ones(disc_interpolates.size()).cuda() if cuda else torch.ones(
                                  disc_interpolates.size()),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    gradients = gradients.view(gradients.size(0), -1)

    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * 10
    return gradient_penalty

### Sample Image function

---

Store generated images every 10 epochs

In [22]:
def sample_image(n_row, epoch_done):
    noise = torch.randn(opt.n_classes * n_row, 100)
    labels = torch.zeros(10, 1)
    for i in range(9):
        temp = torch.ones(10, 1) + i
        labels = torch.cat([labels, temp], 0)
    noise = noise.view(-1, 100, 1, 1)

    onehot_label = torch.zeros(100, 10)
    onehot_label.scatter_(1, labels.type(torch.LongTensor), 1)
    onehot_label = onehot_label.view(-1, 10, 1, 1)

    noise, onehot_label = Variable(noise.cuda(), volatile=True), Variable(onehot_label.cuda(), volatile=True)
    gen_imgs = generator(noise, onehot_label)
    save_image(gen_imgs.data, "/content/drive/My Drive/Summer Project/" + opt.name + "_images/%d.png" % epoch_done, nrow=n_row, normalize=True)

    return noise

In [23]:
def sample_image_S(n_row, epoch_done):
    noise = torch.randn(opt.n_classes * n_row, 100)
    labels = torch.zeros(10, 1)
    for i in range(9):
        temp = torch.ones(10, 1) + i
        labels = torch.cat([labels, temp], 0)
    noise = noise.view(-1, 100, 1, 1)

    onehot_label = torch.zeros(100, 10)
    onehot_label.scatter_(1, labels.type(torch.LongTensor), 1)
    onehot_label = onehot_label.view(-1, 10, 1, 1)

    noise, onehot_label = Variable(noise.cuda(), volatile=True), Variable(onehot_label.cuda(), volatile=True)
    gen_imgs = generator_S(noise, onehot_label)
    save_image(gen_imgs.data, "/content/drive/My Drive/Summer Project/SGAN_" + opt.name + "_images/%d.png" % epoch_done, nrow=n_row, normalize=True)

    return noise

# **Train Generator and Discriminator**

In [24]:
# label preprocess
onehot = torch.zeros(10, 10)
onehot = onehot.scatter_(1, torch.LongTensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]).view(10,1), 1).view(10, 10, 1, 1)
fill = torch.zeros([10, 10, opt.img_size, opt.img_size])
for i in range(10):
    fill[i, i, :, :] = 1


## Train Proposed Model

In [106]:
for epoch in range(opt.n_epochs+1):

    # learning rate decay
    # if epoch == 100:
    #     optimizer_G.param_groups[0]['lr'] /= 10
    #     optimizer_D.param_groups[0]['lr'] /= 10
    for i, (imgs, labels) in enumerate(dataloader):

        # batch_size = imgs.shape[0]
        # valid = torch.zeros(batch_size, opt.n_classes*2)
        # fake = torch.zeros(batch_size, opt.n_classes*2)

        # valid[:,0:10] = 1.0 - torch.Tensor(batch_size, 1).uniform_(0.0, 0.1)
        # valid[:,10:20] = torch.Tensor(batch_size, 1).uniform_(0.0, 0.1)
        # fake[:,0:10] = torch.Tensor(batch_size, 1).uniform_(0.0, 0.1)
        # fake[:,10:20] = 1.0 - torch.Tensor(batch_size, 1).uniform_(0.0, 0.1)
        batch_size = imgs.shape[0]
        valid = torch.zeros(batch_size, opt.n_classes)
        fake = torch.zeros(batch_size, opt.n_classes)

        onehot_ = onehot[labels].reshape(batch_size, 10)

        valid_ = torch.cat((onehot_, valid), dim=1)
        fake_ = torch.cat((fake, onehot_), dim=1)

 
        # # flip labels
        # s = np.max(2, int(opt.flipped_prop*batch_size))
        # flipped_idx = np.random.choice(np.arange(batch_size), size=s)
        # valid[:, flipped_idx] = 1 - valid[:, flipped_idx]

        # flipped_idx = np.random.choice(np.arange(batch_size), size=s)
        # fake[:, flipped_idx] = 1 - fake[:, flipped_idx]

        valid_, fake_ = Variable(valid_.cuda()), Variable(fake_.cuda())

        loss_D = discriminator_train(batch_size, imgs, labels, valid_, fake_)
        loss_G = generator_train(batch_size, valid_, imgs, labels)


        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), loss_D, loss_G)
        )
    if epoch % 10 == 0:
      noise = sample_image(n_row=10, epoch_done = epoch)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1614: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[Epoch 0/200] [Batch 0/40] [D loss: 0.002122] [G loss: 0.595411]
[Epoch 0/200] [Batch 1/40] [D loss: 0.002665] [G loss: 0.616855]
[Epoch 0/200] [Batch 2/40] [D loss: 0.003683] [G loss: 0.564915]
[Epoch 0/200] [Batch 3/40] [D loss: 0.007109] [G loss: 0.600889]
[Epoch 0/200] [Batch 4/40] [D loss: 0.006382] [G loss: 0.576445]
[Epoch 0/200] [Batch 5/40] [D loss: 0.005419] [G loss: 0.548372]
[Epoch 0/200] [Batch 6/40] [D loss: 0.006139] [G loss: 0.502144]
[Epoch 0/200] [Batch 7/40] [D loss: 0.004203] [G loss: 0.525764]
[Epoch 0/200] [Batch 8/40] [D loss: 0.004553] [G loss: 0.567214]
[Epoch 0/200] [Batch 9/40] [D loss: 0.004663] [G loss: 0.542666]
[Epoch 0/200] [Batch 10/40] [D loss: 0.004251] [G loss: 0.457830]
[Epoch 0/200] [Batch 11/40] [D loss: 0.007639] [G loss: 0.477750]
[Epoch 0/200] [Batch 12/40] [D loss: 0.008069] [G loss: 0.443790]
[Epoch 0/200] [Batch 13/40] [D loss: 0.006115] [G loss: 0.519577]
[Epoch 0/200] [Batch 14/40] [D loss: 0.007182] [G loss: 0.520228]
[Epoch 0/200] [Batch

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  del sys.path[0]


流式输出内容被截断，只能显示最后 5000 行内容。
[Epoch 76/200] [Batch 1/40] [D loss: 0.177017] [G loss: 0.589038]
[Epoch 76/200] [Batch 2/40] [D loss: 0.048093] [G loss: 0.495136]
[Epoch 76/200] [Batch 3/40] [D loss: 0.059659] [G loss: 0.501549]
[Epoch 76/200] [Batch 4/40] [D loss: 0.034852] [G loss: 0.496742]
[Epoch 76/200] [Batch 5/40] [D loss: 0.018430] [G loss: 0.588615]
[Epoch 76/200] [Batch 6/40] [D loss: 0.020139] [G loss: 0.506610]
[Epoch 76/200] [Batch 7/40] [D loss: 0.017043] [G loss: 0.493436]
[Epoch 76/200] [Batch 8/40] [D loss: 0.009590] [G loss: 0.645186]
[Epoch 76/200] [Batch 9/40] [D loss: 0.006652] [G loss: 0.579060]
[Epoch 76/200] [Batch 10/40] [D loss: 0.006214] [G loss: 0.609097]
[Epoch 76/200] [Batch 11/40] [D loss: 0.003997] [G loss: 0.513964]
[Epoch 76/200] [Batch 12/40] [D loss: 0.006166] [G loss: 0.587173]
[Epoch 76/200] [Batch 13/40] [D loss: 0.004311] [G loss: 0.606611]
[Epoch 76/200] [Batch 14/40] [D loss: 0.003859] [G loss: 0.659636]
[Epoch 76/200] [Batch 15/40] [D loss: 0.0103

## Train SGAN Model

In [89]:
for epoch in range(opt.n_epochs+1):

    # learning rate decay
    # if epoch == 100:
    #     optimizer_G.param_groups[0]['lr'] /= 10
    #     optimizer_D.param_groups[0]['lr'] /= 10
    for i, (imgs, labels) in enumerate(dataloader):

        # batch_size = imgs.shape[0]
        # valid = torch.zeros(batch_size, opt.n_classes*2)
        # fake = torch.zeros(batch_size, opt.n_classes*2)

        # valid[:,0:10] = 1.0 - torch.Tensor(batch_size, 1).uniform_(0.0, 0.1)
        # valid[:,10:20] = torch.Tensor(batch_size, 1).uniform_(0.0, 0.1)
        # fake[:,0:10] = torch.Tensor(batch_size, 1).uniform_(0.0, 0.1)
        # fake[:,10:20] = 1.0 - torch.Tensor(batch_size, 1).uniform_(0.0, 0.1)
        batch_size = imgs.shape[0]
        valid = torch.zeros(batch_size, 1)
        pad = torch.zeros(batch_size, opt.n_classes)
        fake = torch.ones(batch_size, 1)

        onehot_ = onehot[labels].reshape(batch_size, 10)

        valid_ = torch.cat((valid ,onehot_), dim=1)
        fake_ = torch.cat((fake, pad), dim=1)

 
        # # flip labels
        # s = np.max(2, int(opt.flipped_prop*batch_size))
        # flipped_idx = np.random.choice(np.arange(batch_size), size=s)
        # valid[:, flipped_idx] = 1 - valid[:, flipped_idx]

        # flipped_idx = np.random.choice(np.arange(batch_size), size=s)
        # fake[:, flipped_idx] = 1 - fake[:, flipped_idx]

        valid_, fake_ = Variable(valid_.cuda()), Variable(fake_.cuda())

        loss_D = discriminator_S_train(batch_size, imgs, labels, valid_, fake_)
        loss_G = generator_S_train(batch_size, valid_, imgs, labels)


        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), loss_D, loss_G)
        )
    if epoch % 10 == 0:
      noise = sample_image_S(n_row=10, epoch_done = epoch)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1614: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[Epoch 0/200] [Batch 0/40] [D loss: 1.529616] [G loss: 0.230391]
[Epoch 0/200] [Batch 1/40] [D loss: 0.441915] [G loss: 0.527081]
[Epoch 0/200] [Batch 2/40] [D loss: 0.240315] [G loss: 0.457650]
[Epoch 0/200] [Batch 3/40] [D loss: 0.149470] [G loss: 0.513890]
[Epoch 0/200] [Batch 4/40] [D loss: 0.095950] [G loss: 0.582103]
[Epoch 0/200] [Batch 5/40] [D loss: 0.077980] [G loss: 0.627215]
[Epoch 0/200] [Batch 6/40] [D loss: 0.077741] [G loss: 0.623363]
[Epoch 0/200] [Batch 7/40] [D loss: 0.084527] [G loss: 0.647279]
[Epoch 0/200] [Batch 8/40] [D loss: 0.075226] [G loss: 0.567149]
[Epoch 0/200] [Batch 9/40] [D loss: 0.096953] [G loss: 0.873622]
[Epoch 0/200] [Batch 10/40] [D loss: 0.137260] [G loss: 0.171549]
[Epoch 0/200] [Batch 11/40] [D loss: 0.363320] [G loss: 1.389455]
[Epoch 0/200] [Batch 12/40] [D loss: 0.507508] [G loss: 0.550122]
[Epoch 0/200] [Batch 13/40] [D loss: 0.142170] [G loss: 0.298943]
[Epoch 0/200] [Batch 14/40] [D loss: 0.249798] [G loss: 0.936494]
[Epoch 0/200] [Batch

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  del sys.path[0]


流式输出内容被截断，只能显示最后 5000 行内容。
[Epoch 76/200] [Batch 1/40] [D loss: 0.061184] [G loss: 0.930422]
[Epoch 76/200] [Batch 2/40] [D loss: 0.025426] [G loss: 0.878683]
[Epoch 76/200] [Batch 3/40] [D loss: 0.039897] [G loss: 0.791267]
[Epoch 76/200] [Batch 4/40] [D loss: 0.020252] [G loss: 0.953226]
[Epoch 76/200] [Batch 5/40] [D loss: 0.027022] [G loss: 0.961065]
[Epoch 76/200] [Batch 6/40] [D loss: 0.017692] [G loss: 0.940662]
[Epoch 76/200] [Batch 7/40] [D loss: 0.023158] [G loss: 0.793419]
[Epoch 76/200] [Batch 8/40] [D loss: 0.017676] [G loss: 0.903739]
[Epoch 76/200] [Batch 9/40] [D loss: 0.017426] [G loss: 0.889261]
[Epoch 76/200] [Batch 10/40] [D loss: 0.011795] [G loss: 0.807695]
[Epoch 76/200] [Batch 11/40] [D loss: 0.021081] [G loss: 0.761055]
[Epoch 76/200] [Batch 12/40] [D loss: 0.024038] [G loss: 0.820635]
[Epoch 76/200] [Batch 13/40] [D loss: 0.031601] [G loss: 0.798897]
[Epoch 76/200] [Batch 14/40] [D loss: 0.061828] [G loss: 0.579402]
[Epoch 76/200] [Batch 15/40] [D loss: 0.0263

## Store trained generator and discriminator

In [75]:
generator.eval()
torch.save(generator, "/content/drive/My Drive/Summer Project/model/" + opt.name + "/" + "generator" + str(opt.sample_size) + ".pth")
torch.save(discriminator, "/content/drive/My Drive/Summer Project/model/" + opt.name + "/" + "discriminator" + str(opt.sample_size) +".pth")


In [90]:
generator_S.eval()
torch.save(generator_S, "/content/drive/My Drive/Summer Project/model/" + opt.name + "/" + "generatorS" + str(opt.sample_size) + ".pth")
torch.save(discriminator_S, "/content/drive/My Drive/Summer Project/model/" + opt.name + "/" + "discriminatorS" + str(opt.sample_size) +".pth")

### Generate images using trained generator

In [ ]:
def gen_images(Name):
  index = np.zeros(10)
  for k in range(opt.n_epochs*10):
    noise = torch.randn(opt.n_classes ** 2, 100)
    labels = torch.zeros(10, 1)
    for i in range(9):
        temp = torch.ones(10, 1) + i
        labels = torch.cat([labels, temp], 0)
    noise = noise.view(-1, 100, 1, 1)

    onehot_label = torch.zeros(100, 10)
    onehot_label.scatter_(1, labels.type(torch.LongTensor), 1)
    onehot_label = onehot_label.view(-1, 10, 1, 1)

    noise, onehot_label = Variable(noise.cuda(), volatile=True), Variable(onehot_label.cuda(), volatile=True)
    gen_imgs = generator(noise, onehot_label)

    path = "/content/drive/My Drive/Summer Project/gen_images/" + opt.name + "/" + str(opt.sample_size)
    # print(labels.shape)
    # print(gen_imgs.shape)
    # plt.imshow(gen_imgs[0].data.cpu().detach().numpy().reshape(32,32,3))
    # print(labels[0])
    # print(dsadsa)
    for j in range(10**2):
      save_image(gen_imgs[j].data, path + "/%d/%d.png" % (labels[j], index[j%10]), normalize=True)
      index[j%10]+=1
    
    # save_image(gen_imgs[10].data, "/content/drive/My Drive/Summer Project/mnist_genimages/%d/%d.png" % (0, i), normalize=True)

In [ ]:
gen_images(opt.name)

## Load Exist Generator and Discriminator

In [141]:
generator = torch.load("/content/drive/My Drive/Summer Project/model/" + opt.name + "/" + "generator" + str(opt.sample_size) + ".pth")
discriminator = torch.load("/content/drive/My Drive/Summer Project/model/" + opt.name + "/" + "discriminator" + str(opt.sample_size) +".pth")

generator.cuda()
discriminator.cuda()
generator.eval()
discriminator.eval()

Discriminator(
  (conv1_1): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv1_2): Conv2d(10, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2_bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv3_bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(512, 20, kernel_size=(4, 4), stride=(1, 1))
)

In [142]:
sample_image(10,1000)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1614: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


tensor([[[[-0.2620]],

         [[-0.1759]],

         [[-0.8975]],

         ...,

         [[ 0.3266]],

         [[-0.9429]],

         [[ 0.2808]]],


        [[[ 1.0981]],

         [[ 0.8327]],

         [[-0.5072]],

         ...,

         [[-0.8997]],

         [[-1.3575]],

         [[-1.0463]]],


        [[[-0.7898]],

         [[ 0.9555]],

         [[ 0.9074]],

         ...,

         [[-1.2497]],

         [[ 1.5540]],

         [[ 0.5293]]],


        ...,


        [[[ 0.3671]],

         [[-2.2065]],

         [[-0.0382]],

         ...,

         [[-1.8052]],

         [[ 0.3867]],

         [[ 0.4223]]],


        [[[ 0.6022]],

         [[-1.8874]],

         [[-0.1330]],

         ...,

         [[-0.6047]],

         [[ 1.2167]],

         [[-1.0400]]],


        [[[-0.2495]],

         [[-1.1210]],

         [[-1.0918]],

         ...,

         [[-0.7403]],

         [[ 1.7138]],

         [[ 0.0120]]]], device='cuda:0')

# **Create Second Generator**

---
The second generator takes the images generated by the first generator as input to produce a clearer images. The discriminator is the same one as before.


In [ ]:
# input (batch_size, 3, 32, 32)
# output (batch_size, 3, 32, 32)

# class Generator_2(nn.Module):
#     # initializers
#     def __init__(self, d=128):
#         super(Generator_2, self).__init__()
#         self.conv1_1 = nn.Conv2d(opt.channels, int(d/2), 4, 2, 1) # 64
#         self.conv1_2 = nn.Conv2d(10, int(d/2), 4, 2, 1) # 64
#         self.conv2 = nn.Conv2d(d, d*2, 4, 2, 1) 
#         self.conv2_bn = nn.BatchNorm2d(d*2) # 32
#         self.conv3 = nn.Conv2d(d*2, d*4, 4, 2, 1)
#         self.conv3_bn = nn.BatchNorm2d(d*4) # 16
#         self.conv4 = nn.Conv2d(d * 4, d*8, 4, 1, 0)
#         self.conv4_bn = nn.BatchNorm2d(d*8) # 
        
#         self.deconv1 = nn.ConvTranspose2d(d*8, d*4, 4, 1, 0)
#         self.deconv1_bn = nn.BatchNorm2d(d*4) # 128
#         self.deconv2 = nn.ConvTranspose2d(d*4, d*2, 4, 2, 1)
#         self.deconv2_bn = nn.BatchNorm2d(d*2) # 128
#         self.deconv3 = nn.ConvTranspose2d(d*2, d, 4, 2, 1)
#         self.deconv3_bn = nn.BatchNorm2d(d) # 256
#         self.deconv4 = nn.ConvTranspose2d(d,  opt.channels, 4, 2, 1) 

#     # weight_init
#     def weight_init(self, mean, std):
#         for m in self._modules:
#             normal_init(self._modules[m], mean, std)

#     # forward method
#     def forward(self, imgs, labels):
#         x = F.leaky_relu(self.conv1_1(imgs), 0.2) # 16
#         # print("1",x.shape)
#         y = F.leaky_relu(self.conv1_2(labels), 0.2) # 16
#         # print("2",y.shape)
#         x = torch.cat([x, y], 1) # 16
#         # print("3",x.shape) 
#         x = F.leaky_relu(self.conv2_bn(self.conv2(x)), 0.2) # 8
#         # print("4",x.shape)
#         x = F.leaky_relu(self.conv3_bn(self.conv3(x)), 0.2) # 4
#         # print("5",x.shape)
#         x = F.leaky_relu(self.conv4_bn(self.conv4(x)), 0.2) # 1
#         # print("6",x.shape)

#         x = F.relu(self.deconv1_bn(self.deconv1(x))) # 4
#         # print(x.shape)
#         x = F.relu(self.deconv2_bn(self.deconv2(x))) # 8
#         # print(x.shape)
#         x = F.relu(self.deconv3_bn(self.deconv3(x))) # 16
#         # print(x.shape)
#         # x = F.relu(self.deconv4_bn(self.deconv4(x)))
#         x = F.tanh(self.deconv4(x))
#         # print(x.shape)
#         # x = F.relu(self.deconv5_bn(self.deconv5(x)))
#         # x = F.tanh(self.deconv6(x))
#         # x = F.relu(self.deconv4_bn(self.deconv4(x)))
#         # x = F.tanh(self.deconv5(x))

#         return x

In [ ]:
# input (batch_size, 3, 32, 32)
# output (batch_size, 3, 32, 32)
class ResidualBlock(nn.Module):
    def __init__(self, dim_in, dim_out):
        super(ResidualBlock, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(dim_in, dim_out, kernel_size=3, stride=1, padding=1, bias=False),
            nn.InstanceNorm2d(dim_out, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(dim_out, dim_out, kernel_size=3, stride=1, padding=1, bias=False),
            nn.InstanceNorm2d(dim_out, affine=True, track_running_stats=True))

    def forward(self, x):
        return x + self.main(x)

class Discriminator_2(nn.Module):
    # initializers
    def __init__(self, d=128):
        super(Discriminator_2, self).__init__()
        self.conv = nn.Conv2d(opt.channels, int(d/2), 7, 1, 3)
        self.conv1 = nn.Conv2d(int(d/2), d, 4, 2, 1) # 32
        self.conv1_bn = nn.BatchNorm2d(d)
        self.conv2 = nn.Conv2d(d, d*2, 4, 2, 1) 
        self.conv2_bn = nn.BatchNorm2d(d*2) # 16

        self.res1 = ResidualBlock(d*2,d*2)
        self.res2 = ResidualBlock(d*2,d*2)
        self.res3 = ResidualBlock(d*2,d*2)
        self.res4 = ResidualBlock(d*2,d*2)
        self.res5 = ResidualBlock(d*2,d*2)
        self.res6 = ResidualBlock(d*2,d*2)
        self.res7 = ResidualBlock(d*2,d*2)
        self.res8 = ResidualBlock(d*2,d*2)
        self.res9 = ResidualBlock(d*2,d*2)
 
        self.deconv1 = nn.ConvTranspose2d(d*2, d, 4, 2, 1)    
        self.deconv1_bn = nn.BatchNorm2d(d) # 128
        self.deconv2 = nn.ConvTranspose2d(d, int(d/2), 4, 2, 1)
        self.deconv2_bn = nn.BatchNorm2d(int(d/2)) # 128 int(d/2)
        self.deconv3 = nn.ConvTranspose2d(int(d/2), opt.channels, 7, 1, 3)


    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    # forward method
    def forward(self, imgs):
        x = F.relu(self.conv(imgs)) # 16 x = F.leaky_relu(self.conv(imgs), 0.2) # 16
        # print("1",x.shape)
        x = F.relu(self.conv1_bn(self.conv1(x))) # 8
        # print("2",x.shape)
        x = F.relu(self.conv2_bn(self.conv2(x))) # 4
        # print("3",x.shape)
        x = self.res1(x) # 1
        # # print("4",x.shape)
        x = self.res2(x) # 1
        x = self.res3(x) # 1
        # x = self.res4(x) # 1
        # # print("4",x.shape)
        # x = self.res5(x) # 1
        # x = self.res6(x) # 1
        # x = self.res7(x) # 1
        # # print("4",x.shape)
        # x = self.res8(x) # 1
        # x = self.res9(x) # 1

        x = F.relu(self.deconv1_bn(self.deconv1(x))) # 4
        # print(x.shape)
        x = F.relu(self.deconv2_bn(self.deconv2(x))) # 8
        # print(x.shape)
        # x = F.relu(self.deconv4_bn(self.deconv4(x)))
        x = F.tanh(self.deconv3(x))
        # print(x.shape)
        # x = F.relu(self.deconv5_bn(self.deconv5(x)))
        # x = F.tanh(self.deconv6(x))
        # x = F.relu(self.deconv4_bn(self.deconv4(x)))
        # x = F.tanh(self.deconv5(x))

        return x

In [ ]:
discriminator_2 = Discriminator_2(128)
# discriminator_2 = Discriminator(128)
# generator_2.weight_init(mean=0.0, std=0.02)
discriminator_2.weight_init(mean=0.0, std=0.02)
if cuda:
    # generator_2.cuda()
    discriminator_2.cuda()
# optimizer_G_2 = torch.optim.Adam(generator_2.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D_2 = torch.optim.Adam(discriminator_2.parameters(), lr=0.0005, betas=(opt.b1, opt.b2))
k = 0

## Train Function and Sample Function


In [ ]:
def generator_train_2(batch_size, valid, gen_imgs, labels):
    optimizer_G.zero_grad()

    noise = torch.randn((batch_size, 100)).view(-1, 100, 1, 1)
    # noise = mean[labels].reshape(8,1) + noise * std[labels].reshape(8,1)
    # noise = noise.view(-1, 100, 1, 1)

    # noise = torch.normal(mean, std)
    # fake_labels = (torch.rand(batch_size, 1) * 10).type(torch.LongTensor).squeeze()
    # fake_labels = (torch.rand(batch_size, 1) * 10).type(torch.LongTensor).reshape(1)
    gen_fake_labels = onehot[labels]
    dis_fake_labels = fill[labels]
    noise, gen_fake_labels, dis_fake_labels = Variable(noise.cuda()), Variable(gen_fake_labels.cuda()), Variable(dis_fake_labels.cuda())

    gen_imgs = generator(noise, gen_fake_labels)
    # plt.imshow(gen_imgs[0].cpu().detach().numpy().reshape(32,32))
    # print(gen_fake_labels[0])
    # print(Dsaas)
    gen_imgs_2 = discriminator_2(gen_imgs)
    v, _ = discriminator(gen_imgs_2, dis_fake_labels)
    v = v.squeeze()

    # validity, _ = discriminator_2(gen_imgs_2, dis_fake_labels)
    # validity = validity.squeeze()
    # regularization = torch.norm(imgs.cpu() - gen_imgs.cpu())
    # regularization = Variable(regularization.cuda())
    # loss_G = adversarial_loss(validity, valid)
    # loss_G = adversarial_loss(validity, valid)
    loss_G = torch.norm(gen_imgs_2 - gen_imgs) + 10 * adversarial_loss(v, valid)
    loss_G.backward()
    optimizer_G.step()
    # plt.imshow(gen_imgs[0].cpu().detach().numpy().reshape(32,32,3))
    # plt.show()
    l = loss_G
    return l

In [ ]:
def discriminator_train_2(batch_size, imgs, labels, valid, fake, k):
    optimizer_D_2.zero_grad()
    real_labels = fill[labels]
    imgs, real_labels = Variable(imgs.cuda()), Variable(real_labels.cuda())
    # validity_real, hidden1 = discriminator_2(imgs, real_labels)
    # validity_real = validity_real.squeeze()
    # d_real_loss = adversarial_loss(validity_real, valid)

    noise = torch.randn((batch_size, 100)).view(-1, 100, 1, 1)
    # fake_labels = (torch.rand(batch_size, 1) * 10).type(torch.LongTensor).squeeze()
    # fake_labels = (torch.rand(batch_size, 1) * 10).type(torch.LongTensor).reshape(1)
    gen_fake_labels = onehot[labels]
    dis_fake_labels = fill[labels]
    noise, gen_fake_labels, dis_fake_labels = Variable(noise.cuda()), Variable(gen_fake_labels.cuda()), Variable(dis_fake_labels.cuda())
    gen_imgs = generator(noise, gen_fake_labels)
    gen_imgs_2 = discriminator_2(gen_imgs)
    
    gen_real = discriminator_2(imgs)
    
    # validity_fake, hidden2 = discriminator_2(gen_imgs_2, real_labels)
    # validity_fake = validity_fake.squeeze()
    # print(validity_fake)
    
    # d_fake_loss = adversarial_loss(validity_fake, fake)
    gp = calc_gradient_penalty(discriminator, imgs, gen_imgs_2, batch_size, labels)
    # print(d_fake_loss)
    # print(d_real_loss)
    # print(dsada)
    # gp = calc_gradient_penalty(generator_2, imgs, gen_imgs, batch_size, real_labels)
    # regularization = torch.norm((hidden1.mean() - hidden2.mean()).cuda())
    # loss_D = (d_real_loss + d_fake_loss) / 2 + 10 * gp
    # local = k 
    loss_D2 = torch.norm(imgs - gen_real) + torch.norm(gen_imgs_2 - gen_imgs)
    loss_D2.backward()
    # k += 0.01 * (0.75 * l2 - l)
    # k = min(max(k,0),1)
    optimizer_D_2.step()
    return loss_D2

In [ ]:
def sample_image_2(n_row, epoch_done):
    noise = torch.randn(opt.n_classes * n_row, 100)
    labels = torch.zeros(10, 1)
    for i in range(9):
        temp = torch.ones(10, 1) + i
        labels = torch.cat([labels, temp], 0)
    noise = noise.view(-1, 100, 1, 1)

    onehot_label = torch.zeros(100, 10)
    onehot_label.scatter_(1, labels.type(torch.LongTensor), 1)
    onehot_label = onehot_label.view(-1, 10, 1, 1)
    gen_labels = fill[labels.type(torch.LongTensor).squeeze()]
    noise, onehot_label, gen_labels = Variable(noise.cuda(), volatile=True), Variable(onehot_label.cuda(), volatile=True), Variable(gen_labels.cuda(), volatile=True)
    gen_imgs = generator(noise, onehot_label)

    gen_imgs_2 = generator_2(gen_imgs)
    save_image(gen_imgs_2.data, "/content/drive/My Drive/Summer Project/" + opt.name + "_images/%d.png" % epoch_done, nrow=n_row, normalize=True)

    return noise

## Train the Second Generator

In [ ]:
for epoch in range(opt.n_epochs+1):

    # learning rate decay
    # if epoch == 100:
    #     optimizer_G.param_groups[0]['lr'] /= 10
    #     optimizer_D.param_groups[0]['lr'] /= 10
    for i, (imgs, labels) in enumerate(dataloader):
        
        batch_size = imgs.shape[0]
        valid = torch.zeros(batch_size, opt.n_classes*2)
        fake = torch.zeros(batch_size, opt.n_classes*2)

        valid[:,0:10] = 1.0 - torch.Tensor(batch_size, 1).uniform_(0.0, 0.1)
        valid[:,10:20] = torch.Tensor(batch_size, 1).uniform_(0.0, 0.1)
        fake[:,0:10] = torch.Tensor(batch_size, 1).uniform_(0.0, 0.1)
        fake[:,10:20] = 1.0 - torch.Tensor(batch_size, 1).uniform_(0.0, 0.1)
        # # flip labels
        # s = np.max(2, int(opt.flipped_prop*batch_size))
        # flipped_idx = np.random.choice(np.arange(batch_size), size=s)
        # valid[:, flipped_idx] = 1 - valid[:, flipped_idx]

        # flipped_idx = np.random.choice(np.arange(batch_size), size=s)
        # fake[:, flipped_idx] = 1 - fake[:, flipped_idx]

        valid, fake = Variable(valid.cuda()), Variable(fake.cuda())
        loss_G = generator_train_2(batch_size, valid, imgs, labels)
        loss_D = discriminator_train_2(batch_size, imgs, labels, valid, fake,k)
        k += 0.01 * (0.75 * loss_D - loss_G)
        k = min(max(k,0),1)
        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f] [k: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), loss_D, loss_G, k)
        )
    if epoch % 10 == 0:
      noise = sample_image(n_row=10, epoch_done = epoch)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1614: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[Epoch 0/200] [Batch 0/8] [D loss: 905.337097] [G loss: 538.786133] [k: 1.000000]
[Epoch 0/200] [Batch 1/8] [D loss: 710.038208] [G loss: 410.084717] [k: 1.000000]
[Epoch 0/200] [Batch 2/8] [D loss: 541.129639] [G loss: 322.891357] [k: 1.000000]
[Epoch 0/200] [Batch 3/8] [D loss: 469.893066] [G loss: 281.213623] [k: 1.000000]
[Epoch 0/200] [Batch 4/8] [D loss: 409.840576] [G loss: 244.870010] [k: 1.000000]
[Epoch 0/200] [Batch 5/8] [D loss: 401.304840] [G loss: 242.825974] [k: 1.000000]
[Epoch 0/200] [Batch 6/8] [D loss: 453.797424] [G loss: 265.524902] [k: 1.000000]
[Epoch 0/200] [Batch 7/8] [D loss: 318.934723] [G loss: 198.652405] [k: 1.000000]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  del sys.path[0]


[Epoch 1/200] [Batch 0/8] [D loss: 328.750305] [G loss: 199.186890] [k: 1.000000]
[Epoch 1/200] [Batch 1/8] [D loss: 298.000305] [G loss: 183.699005] [k: 1.000000]
[Epoch 1/200] [Batch 2/8] [D loss: 305.886993] [G loss: 186.151367] [k: 1.000000]
[Epoch 1/200] [Batch 3/8] [D loss: 310.685730] [G loss: 198.862717] [k: 1.000000]
[Epoch 1/200] [Batch 4/8] [D loss: 280.960632] [G loss: 178.266785] [k: 1.000000]
[Epoch 1/200] [Batch 5/8] [D loss: 277.833740] [G loss: 171.286270] [k: 1.000000]
[Epoch 1/200] [Batch 6/8] [D loss: 332.468353] [G loss: 205.803925] [k: 1.000000]
[Epoch 1/200] [Batch 7/8] [D loss: 253.642487] [G loss: 162.806885] [k: 1.000000]
[Epoch 2/200] [Batch 0/8] [D loss: 264.355530] [G loss: 163.047729] [k: 1.000000]
[Epoch 2/200] [Batch 1/8] [D loss: 255.635956] [G loss: 161.429688] [k: 1.000000]
[Epoch 2/200] [Batch 2/8] [D loss: 237.409424] [G loss: 151.475861] [k: 1.000000]
[Epoch 2/200] [Batch 3/8] [D loss: 249.806870] [G loss: 157.754593] [k: 1.000000]
[Epoch 2/200] [B

In [ ]:
 for i, (imgs, labels) in enumerate(dataloader):  
    real = labels 
    imgs, labels = Variable(imgs.cuda()), Variable(labels.cuda())
    noise = torch.randn(opt.n_classes * 10, 100)
    labels = torch.zeros(10, 1)
    for i in range(9):
        temp = torch.ones(10, 1) + i
        labels = torch.cat([labels, temp], 0)
    noise = noise.view(-1, 100, 1, 1)

    onehot_label = torch.zeros(100, 10)
    onehot_label.scatter_(1, labels.type(torch.LongTensor), 1)
    onehot_label = onehot_label.view(-1, 10, 1, 1)
    gen_labels = fill[labels.type(torch.LongTensor).squeeze()]
    noise, onehot_label, gen_labels = Variable(noise.cuda(), volatile=True), Variable(onehot_label.cuda(), volatile=True), Variable(gen_labels.cuda(), volatile=True)
    gen_imgs = generator(noise, onehot_label)
    v, _ = discriminator(gen_imgs, gen_labels)
    gen_imgs_2 = discriminator_2(gen_imgs)
    save_image(imgs.data, "/content/drive/My Drive/Summer Project/%d.png" % 1, normalize=True)
    save_image(gen_imgs_2.data, "/content/drive/My Drive/Summer Project/%d.png" % 2, normalize=True)
    print(real)
    print(dsadas)

tensor([3, 7, 3, 2, 8, 5, 4, 5, 7, 4, 8, 2, 5, 5, 6, 3, 6, 1, 7, 1, 6, 7, 5, 9,
        9, 9, 2, 5, 2, 0, 4, 4, 3, 6, 5, 0, 0, 1, 6, 2, 9, 0, 8, 9, 0, 8, 6, 6,
        7, 1, 6, 2, 2, 1, 3, 4, 5, 6, 0, 9, 8, 1, 5, 9, 5, 1, 2, 8, 0, 6, 9, 1,
        7, 7, 5, 2, 6, 7, 2, 0, 4, 5, 1, 0, 6, 9, 3, 4, 9, 0, 6, 4, 2, 2, 0, 3,
        8, 3, 3, 2, 8, 4, 0, 3, 4, 1, 2, 9, 1, 7, 5, 5, 4, 1, 8, 8, 4, 7, 5, 6,
        3, 3, 7, 0, 7, 4, 0, 2])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1614: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


NameError: ignored

# **Clissification**

## Load Local Data

In [ ]:
path = "/content/drive/My Drive/Summer Project/gen_images/" + opt.name + "/" + str(opt.sample_size)
gen_transform = transforms.Compose([
            transforms.ToTensor(), 
            transforms.Normalize([0.5], [0.5])
])
gen_dataset = ImageFolder(path, transform= gen_transform)

print(gen_dataset.class_to_idx)

print(gen_dataset[0][1])# 第一维是第几张图，第二维为1返回label
print(gen_dataset[1][0].shape) # 为0返回图片数据
print(dataloader.dataset[0][0].shape)

In [ ]:
def get_trainloader(Name, real_loader):
  train_images = []
  train_labels = []

  for e in range(opt.n_epochs*10):
    print("epoch:", e)
    noise = torch.randn(opt.n_classes ** 2, 100)
    labels = torch.zeros(10, 1)
    for i in range(9):
        temp = torch.ones(10, 1) + i
        labels = torch.cat([labels, temp], 0)
    noise = noise.view(-1, 100, 1, 1)

    onehot_label = torch.zeros(100, 10)
    onehot_label.scatter_(1, labels.type(torch.LongTensor), 1)
    onehot_label = onehot_label.view(-1, 10, 1, 1)

    noise, onehot_label = Variable(noise.cuda(), volatile=True), Variable(onehot_label.cuda(), volatile=True)
    gen_imgs = generator(noise, onehot_label)
    

    for j in range(10**2):
      # save_image(gen_imgs[25].data, "/content/drive/My Drive/Summer Project/%d.png" % 1, normalize=True)
      train_images.append(gen_imgs[j].data)
      train_labels.append(int(labels[j]))
      # print(int(labels[25]))
      # print(dsadsa)


  for k in range(len(real_loader.dataset.data_tensor)):
    print("real:", k)
    train_images.append(real_loader.dataset.data_tensor[k].cuda())
    train_labels.append(real_loader.dataset.target_tensor[k])


  train_dataset = MyDataset(train_images, train_labels, transform=transforms.Compose(
              [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
          ))

  train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=128,
        shuffle=True,
    )
  return train_dataloader

## Claiisifier Trained with Original Data

### Classifier

In [143]:
class Classifier(nn.Module):
    # initializers
    def __init__(self):
      super(Classifier, self).__init__() 
      self.conv1 = nn.Conv2d(opt.channels, 10, 5)
      self.conv2 = nn.Conv2d(10, 20, 5)
      self.fc1 = nn.Linear(500, 50)
      self.fc2 = nn.Linear(50, 10)

    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    def forward(self, x):
        x = self.conv1(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = F.relu(x)
    
        x = self.conv2(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = F.relu(x)
    
        x = x.view(-1, 500)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.sigmoid(x)

def normal_init(m, mean, std):
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
        m.weight.data.normal_(mean, std)
        m.bias.data.zero_()

### Train

In [144]:
# D = discriminator
# original_model = Discriminator(128)
# original_model.cuda()
# original_model.train()
# optimizer_original = torch.optim.SGD(original_model.parameters(), lr=0.1)
original_model = Classifier()
optimizer_original = torch.optim.SGD(original_model.parameters(), lr=0.1)

classification_loss = torch.nn.CrossEntropyLoss()

if cuda:
    original_model.cuda()
    classification_loss.cuda()

In [145]:
original_model.train()

Classifier(
  (conv1): Conv2d(3, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=500, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [146]:
for epoch in range(opt.n_epochs*2):
    for i, (imgs, labels) in enumerate(dataloader):
        batch_size = imgs.shape[0]
        # Adversarial ground truths
        optimizer_original.zero_grad()

        # Loss measures generator's ability to fool the discriminator
        imgs = Variable(imgs.cuda())
        validity = original_model(imgs)

        loss = classification_loss(validity, labels.cuda())

        loss.backward()

        optimizer_original.step()

    print("Epoch: %d, [loss: %f]"% (epoch, loss.item()))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 0, [loss: 2.304971]
Epoch: 1, [loss: 2.306217]
Epoch: 2, [loss: 2.302450]
Epoch: 3, [loss: 2.298986]
Epoch: 4, [loss: 2.298933]
Epoch: 5, [loss: 2.291830]
Epoch: 6, [loss: 2.208122]
Epoch: 7, [loss: 2.026736]
Epoch: 8, [loss: 1.992306]
Epoch: 9, [loss: 1.838685]
Epoch: 10, [loss: 1.688273]
Epoch: 11, [loss: 1.879264]
Epoch: 12, [loss: 1.688720]
Epoch: 13, [loss: 1.652700]
Epoch: 14, [loss: 1.710317]
Epoch: 15, [loss: 1.715591]
Epoch: 16, [loss: 1.705084]
Epoch: 17, [loss: 1.716402]
Epoch: 18, [loss: 1.573911]
Epoch: 19, [loss: 1.650317]
Epoch: 20, [loss: 1.624186]
Epoch: 21, [loss: 1.586394]
Epoch: 22, [loss: 1.662092]
Epoch: 23, [loss: 1.581007]
Epoch: 24, [loss: 1.582622]
Epoch: 25, [loss: 1.649810]
Epoch: 26, [loss: 1.553364]
Epoch: 27, [loss: 1.606247]
Epoch: 28, [loss: 1.678286]
Epoch: 29, [loss: 1.523852]
Epoch: 30, [loss: 1.526716]
Epoch: 31, [loss: 1.593185]
Epoch: 32, [loss: 1.583163]
Epoch: 33, [loss: 1.565225]
Epoch: 34, [loss: 1.617066]
Epoch: 35, [loss: 1.554150]
Ep

In [147]:
total = 0
acc = 0
original_model.eval()
for i, (imgs, labels) in enumerate(testloader):
    batch_size = imgs.shape[0]
    # Configure input
    real_imgs = Variable(imgs.type(FloatTensor))
    labels = Variable(labels.type(LongTensor))

    validity = original_model(real_imgs)
    result = torch.argmax(validity, axis = 1)
  
    for j in range(batch_size):
      total += 1
      if result[j] == labels[j]:
        acc += 1

print('accuracy: ', acc/total*100)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


accuracy:  84.0542409342348


In [ ]:
# for epoch in range(100):
#     for j, (imgs, labels) in enumerate(dataloader):
#         batch_size = imgs.shape[0]
#         # Adversarial ground truths
#         optimizer_original.zero_grad()

#         fill_labels = fill[labels]
#         fill_labels.zero_()
#         fill_labels = fill_labels.cuda()
#         imgs = Variable(imgs.cuda())
#         validity, _ = original_model(imgs, fill_labels) # 64，20
#         validity = validity.squeeze()

#         # validity = torch.argmax(validity, axis=1).reshape((64,1))
#         result = np.zeros((batch_size, opt.n_classes))

#         result = Variable(FloatTensor(result))
#         for i in range(opt.n_classes):
#           result[:,i:i+1] = validity[:,i:i+1] + validity[:,i+10:i+11]
#         target = onehot[labels].view(-1,10)

#         loss = classification_loss(result, labels.cuda())

#         loss.backward()

#         optimizer_original.step()

#     print("[epoch: %d], [loss: %f]"% (epoch, loss.item()))

In [ ]:
# total = 0
# acc = 0
# original_model.eval()
# for e, (imgs, labels) in enumerate(testloader):
#     batch_size = imgs.shape[0]

#     # Configure input
#     real_imgs = Variable(imgs.type(FloatTensor))
#     labels = Variable(labels.type(LongTensor))

#     fill_labels = (torch.rand(batch_size, 1) * 10).type(torch.LongTensor).squeeze()
    
#     fill_labels = fill[fill_labels]
#     fill_labels.zero_()
#     fill_labels = fill_labels.cuda()
#     validity, _ = original_model(real_imgs, fill_labels) # 64，20
#     validity = validity.squeeze()
#     result = torch.zeros((batch_size, opt.n_classes))

#     for i in range(opt.n_classes):
#       result[:,i:i+1] = validity[:,i:i+1] + validity[:,i+10:i+11]

#     res = torch.argmax(result, axis = 1)
#     for j in range(batch_size):
#       total += 1
#       a = labels[j].cpu()
#       if res[j] == a:
#         acc += 1

# print('accuracy: ', acc/total*100)

## Clissifier trained with Combine Data

### Load Combine Data

In [159]:
def get_trainloader2(Name, real_loader):
  train_images = []
  train_labels = []

  for e in range(opt.n_epochs*10):
    print("epoch:", e)
    noise = torch.randn(opt.n_classes ** 2, 100)
    labels = torch.zeros(10, 1)
    for i in range(9):
        temp = torch.ones(10, 1) + i
        labels = torch.cat([labels, temp], 0)
    noise = noise.view(-1, 100, 1, 1)

    onehot_label = torch.zeros(100, 10)
    onehot_label.scatter_(1, labels.type(torch.LongTensor), 1)
    onehot_label = onehot_label.view(-1, 10, 1, 1)
    dis_fake_labels = fill[labels.type(torch.LongTensor).squeeze()]
    noise, onehot_label, dis_fake_labels = Variable(noise.cuda()), Variable(onehot_label.cuda()),  Variable(dis_fake_labels.cuda())
    gen_imgs = generator(noise, onehot_label)
    
    fill_labels = (torch.rand(100, 1) * 10).type(torch.LongTensor).squeeze()
    
    fill_labels = fill[fill_labels]
    fill_labels.zero_()
    fill_labels = fill_labels.cuda()
    # #get sfot label from model 1
    # validity, _ = original_model(gen_imgs, fill_labels) # 64，20
    # validity = validity.squeeze()

    # result = np.zeros((100, opt.n_classes))

    # result = Variable(FloatTensor(result))
    # for i in range(opt.n_classes):
    #   result[:,i:i+1] = validity[:,i:i+1] + validity[:,i+10:i+11] # soft_labels [64,10]

    # res = torch.argmax(result, axis = 1)

    for j in range(10**2):
      # save_image(gen_imgs[25].data, "/content/drive/My Drive/Summer Project/%d.png" % 1, normalize=True)
      train_images.append(gen_imgs[j].data)
      train_labels.append(int(labels[j]))
      # print(int(labels[25]))
      # print(dsadsa)


  for k in range(len(real_loader.dataset.data_tensor)):
    print("real:", k)
    train_images.append(real_loader.dataset.data_tensor[k].cuda())
    train_labels.append(real_loader.dataset.target_tensor[k])


  train_dataset = MyDataset(train_images, train_labels, transform=transforms.Compose(
              [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
          ))

  train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=128,
        shuffle=True,
    )
  return train_dataloader

In [160]:
train_dataloader = get_trainloader2(opt.name, dataloader)

epoch: 0
epoch: 1
epoch: 2
epoch: 3
epoch: 4
epoch: 5
epoch: 6
epoch: 7
epoch: 8
epoch: 9
epoch: 10
epoch: 11
epoch: 12
epoch: 13
epoch: 14
epoch: 15
epoch: 16
epoch: 17
epoch: 18


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1614: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


流式输出内容被截断，只能显示最后 5000 行内容。
real: 15001
real: 15002
real: 15003
real: 15004
real: 15005
real: 15006
real: 15007
real: 15008
real: 15009
real: 15010
real: 15011
real: 15012
real: 15013
real: 15014
real: 15015
real: 15016
real: 15017
real: 15018
real: 15019
real: 15020
real: 15021
real: 15022
real: 15023
real: 15024
real: 15025
real: 15026
real: 15027
real: 15028
real: 15029
real: 15030
real: 15031
real: 15032
real: 15033
real: 15034
real: 15035
real: 15036
real: 15037
real: 15038
real: 15039
real: 15040
real: 15041
real: 15042
real: 15043
real: 15044
real: 15045
real: 15046
real: 15047
real: 15048
real: 15049
real: 15050
real: 15051
real: 15052
real: 15053
real: 15054
real: 15055
real: 15056
real: 15057
real: 15058
real: 15059
real: 15060
real: 15061
real: 15062
real: 15063
real: 15064
real: 15065
real: 15066
real: 15067
real: 15068
real: 15069
real: 15070
real: 15071
real: 15072
real: 15073
real: 15074
real: 15075
real: 15076
real: 15077
real: 15078
real: 15079
real: 15080
real: 15081
r

In [161]:
# D = discriminator
# combine_model = Discriminator(128)
combine_model = discriminator
combine_model.cuda()
combine_model.train()
optimizer_combine = torch.optim.SGD(combine_model.parameters(), lr=0.1)

In [162]:
for epoch in range(100):
    for j, (imgs, labels) in enumerate(train_dataloader):
        batch_size = imgs.shape[0]
        # Adversarial ground truths
        optimizer_combine.zero_grad()

        fill_labels = fill[labels]
        fill_labels.zero_()
        fill_labels = fill_labels.cuda()
        imgs = Variable(imgs.cuda())
        validity, _ = combine_model(imgs, fill_labels) # 64，20
        validity = validity.squeeze()

        # validity = torch.argmax(validity, axis=1).reshape((64,1))
        result = np.zeros((batch_size, opt.n_classes))

        result = Variable(FloatTensor(result))
        for i in range(opt.n_classes):
          result[:,i:i+1] = validity[:,i:i+1] + validity[:,i+10:i+11]
        target = onehot[labels].view(-1,10)

        loss = classification_loss(result, labels.cuda())

        loss.backward()

        optimizer_combine.step()

        print("[epoch %d: %d/%d], [loss: %f]"% (epoch, j , len(train_dataloader), loss.item()))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


流式输出内容被截断，只能显示最后 5000 行内容。
[epoch 97: 158/1719], [loss: 0.802197]
[epoch 97: 159/1719], [loss: 0.804085]
[epoch 97: 160/1719], [loss: 0.797137]
[epoch 97: 161/1719], [loss: 0.810023]
[epoch 97: 162/1719], [loss: 0.832431]
[epoch 97: 163/1719], [loss: 0.835688]
[epoch 97: 164/1719], [loss: 0.802462]
[epoch 97: 165/1719], [loss: 0.798856]
[epoch 97: 166/1719], [loss: 0.832827]
[epoch 97: 167/1719], [loss: 0.807543]
[epoch 97: 168/1719], [loss: 0.816088]
[epoch 97: 169/1719], [loss: 0.812010]
[epoch 97: 170/1719], [loss: 0.809693]
[epoch 97: 171/1719], [loss: 0.812600]
[epoch 97: 172/1719], [loss: 0.803739]
[epoch 97: 173/1719], [loss: 0.799829]
[epoch 97: 174/1719], [loss: 0.824627]
[epoch 97: 175/1719], [loss: 0.804602]
[epoch 97: 176/1719], [loss: 0.805744]
[epoch 97: 177/1719], [loss: 0.797653]
[epoch 97: 178/1719], [loss: 0.796935]
[epoch 97: 179/1719], [loss: 0.797559]
[epoch 97: 180/1719], [loss: 0.797259]
[epoch 97: 181/1719], [loss: 0.809401]
[epoch 97: 182/1719], [loss: 0.821913

In [163]:
total = 0
acc = 0
combine_model.eval()
for e, (imgs, labels) in enumerate(testloader):
    batch_size = imgs.shape[0]

    # Configure input
    real_imgs = Variable(imgs.type(FloatTensor))
    labels = Variable(labels.type(LongTensor))

    fill_labels = (torch.rand(batch_size, 1) * 10).type(torch.LongTensor).squeeze()
    
    fill_labels = fill[fill_labels]
    fill_labels.zero_()
    fill_labels = fill_labels.cuda()
    validity, _ = combine_model(real_imgs, fill_labels) # 64，20
    validity = validity.squeeze()
    result = torch.zeros((batch_size, opt.n_classes))

    for i in range(opt.n_classes):
      result[:,i:i+1] = validity[:,i:i+1] + validity[:,i+10:i+11]

    res = torch.argmax(result, axis = 1)

    for j in range(batch_size):
      total += 1
      a = labels[j].cpu()
      if res[j] == a:
        acc += 1

print('accuracy: ', acc/total*100)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


accuracy:  84.0158266748617


## Classifier Trained with Hard Label

In [148]:
hard_model = Discriminator(opt.latent_dim)
# hard_model = discriminator
hard_model.cuda()
optimizer_hard = torch.optim.SGD(hard_model.parameters(), lr=0.1)

In [149]:
hard_model.train()

Discriminator(
  (conv1_1): Conv2d(3, 50, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv1_2): Conv2d(10, 50, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(100, 200, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2_bn): BatchNorm2d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(200, 400, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv3_bn): BatchNorm2d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(400, 20, kernel_size=(4, 4), stride=(1, 1))
)

### Train with Hard Label

In [150]:
for epoch in range(opt.n_epochs):
    batch_size = 512
    # Adversarial ground truths
    optimizer_hard.zero_grad()

    # Sample noise and labels as generator input
    noise = torch.randn((batch_size, 100)).view(-1, 100, 1, 1)
    fake_labels = (torch.rand(batch_size, 1) * 10).type(torch.LongTensor).squeeze()
    gen_fake_labels = onehot[fake_labels]   
    dis_fake_labels = fill[fake_labels]
    noise, gen_fake_labels, dis_fake_labels = Variable(noise.cuda()), Variable(gen_fake_labels.cuda()), Variable(dis_fake_labels.cuda())
    gen_imgs = generator(noise, gen_fake_labels)

    # gen_imgs_2 = generator_2(gen_imgs)
    # print(fake_labels[0])
    # plt.imshow(gen_imgs_2[0].cpu().detach().numpy().reshape(32,32))
    # print(dsads)


    fill_labels = (torch.rand(batch_size, 1) * 10).type(torch.LongTensor).squeeze()
    
    fill_labels = fill[fill_labels]
    fill_labels.zero_()
    fill_labels = fill_labels.cuda()

    #get sfot label from model 1
    # validity, _ = original_model(gen_imgs_2, fill_labels) # 64，20
    # validity = validity.squeeze()

    # # print(validity[0])
    # # print(gen_labels[0])
    # # print(fake_labels[0])
    # # print(dasdas)
    # validity = torch.argmax(validity, axis=1).reshape((64,1))
    # result = np.zeros((batch_size, opt.n_classes))

    # result = Variable(FloatTensor(result))
    # for i in range(opt.n_classes):
    #   result[:,i:i+1] = validity[:,i:i+1] + validity[:,i+10:i+11] # hard_model [64,10]


    # train model 2
    validity3, _ = hard_model(gen_imgs, fill_labels)
    validity3 = validity3.squeeze()

    result3 = np.zeros((batch_size, opt.n_classes))

    result3 = Variable(FloatTensor(result3))
    for i in range(opt.n_classes):
      result3[:,i:i+1] = validity3[:,i:i+1] + validity3[:,i+10:i+11] # hard_model [64,10]

    target = gen_fake_labels.view(-1,10)

    # res = torch.argmax(result, axis = 1)
    # loss2 = classification_loss(result2, res)
    # loss2 = classification_loss2(result2, result)
    loss3 = classification_loss(result3, fake_labels.cuda())

    loss3.backward()

    optimizer_hard.step()

    print("[epoch: %d], [loss: %f]"% (epoch, loss3.item()))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1614: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[epoch: 0], [loss: 2.313566]
[epoch: 1], [loss: 2.301199]
[epoch: 2], [loss: 2.250605]
[epoch: 3], [loss: 2.223808]
[epoch: 4], [loss: 2.201161]
[epoch: 5], [loss: 2.187867]
[epoch: 6], [loss: 2.136292]
[epoch: 7], [loss: 2.084076]
[epoch: 8], [loss: 2.076327]
[epoch: 9], [loss: 2.026244]
[epoch: 10], [loss: 1.988106]
[epoch: 11], [loss: 1.978237]
[epoch: 12], [loss: 1.960781]
[epoch: 13], [loss: 1.960819]
[epoch: 14], [loss: 1.917809]
[epoch: 15], [loss: 1.890717]
[epoch: 16], [loss: 1.836375]
[epoch: 17], [loss: 1.795810]
[epoch: 18], [loss: 1.815902]
[epoch: 19], [loss: 1.798061]
[epoch: 20], [loss: 1.750952]
[epoch: 21], [loss: 1.711916]
[epoch: 22], [loss: 1.717106]
[epoch: 23], [loss: 1.690971]
[epoch: 24], [loss: 1.676633]
[epoch: 25], [loss: 1.677218]
[epoch: 26], [loss: 1.629763]
[epoch: 27], [loss: 1.593658]
[epoch: 28], [loss: 1.589689]
[epoch: 29], [loss: 1.563586]
[epoch: 30], [loss: 1.571214]
[epoch: 31], [loss: 1.561383]
[epoch: 32], [loss: 1.495476]
[epoch: 33], [loss: 

### Train with Original Data

In [151]:
for epoch in range(opt.n_epochs):
    for j, (imgs, labels) in enumerate(dataloader):
        batch_size = imgs.shape[0]
        # Adversarial ground truths
        optimizer_hard.zero_grad()

        fill_labels = fill[labels]
        fill_labels.zero_()
        fill_labels = fill_labels.cuda()
        imgs = Variable(imgs.cuda())
        validity3, _ = hard_model(imgs, fill_labels) # 64，20
        validity3 = validity3.squeeze()

        # validity = torch.argmax(validity, axis=1).reshape((64,1))
        result3 = np.zeros((batch_size, opt.n_classes))

        result3 = Variable(FloatTensor(result3))
        for i in range(opt.n_classes):
          result3[:,i:i+1] = validity3[:,i:i+1] + validity3[:,i+10:i+11]
        target = onehot[labels].view(-1,10)

        loss3 = classification_loss(result3, labels.cuda())

        loss3.backward()

        optimizer_hard.step()

    print("[epoch: %d], [loss: %f]"% (epoch, loss3.item()))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[epoch: 0], [loss: 1.158952]
[epoch: 1], [loss: 1.054409]
[epoch: 2], [loss: 1.188312]
[epoch: 3], [loss: 0.960376]
[epoch: 4], [loss: 1.101931]
[epoch: 5], [loss: 0.999128]
[epoch: 6], [loss: 0.863659]
[epoch: 7], [loss: 0.935055]
[epoch: 8], [loss: 0.926430]
[epoch: 9], [loss: 0.868584]
[epoch: 10], [loss: 0.906071]
[epoch: 11], [loss: 0.924286]
[epoch: 12], [loss: 0.980075]
[epoch: 13], [loss: 0.858753]
[epoch: 14], [loss: 0.880822]
[epoch: 15], [loss: 0.827297]
[epoch: 16], [loss: 0.859548]
[epoch: 17], [loss: 0.854124]
[epoch: 18], [loss: 0.926158]
[epoch: 19], [loss: 0.961241]
[epoch: 20], [loss: 0.842340]
[epoch: 21], [loss: 0.833790]
[epoch: 22], [loss: 0.813593]
[epoch: 23], [loss: 0.832442]
[epoch: 24], [loss: 0.869903]
[epoch: 25], [loss: 0.831514]
[epoch: 26], [loss: 0.919043]
[epoch: 27], [loss: 0.811131]
[epoch: 28], [loss: 0.831638]
[epoch: 29], [loss: 0.810628]
[epoch: 30], [loss: 0.855448]
[epoch: 31], [loss: 0.918740]
[epoch: 32], [loss: 0.803180]
[epoch: 33], [loss: 

In [152]:
total = 0
acc = 0
hard_model.eval()
for e, (imgs, labels) in enumerate(testloader):
    batch_size = imgs.shape[0]

    # Configure input
    real_imgs = Variable(imgs.type(FloatTensor))
    labels = Variable(labels.type(LongTensor))

    fill_labels = (torch.rand(batch_size, 1) * 10).type(torch.LongTensor).squeeze()
    
    fill_labels = fill[fill_labels]
    fill_labels.zero_()
    fill_labels = fill_labels.cuda()
    validity, _ = hard_model(real_imgs, fill_labels) # 64，20
    validity = validity.squeeze()
    result = torch.zeros((batch_size, opt.n_classes))

    for i in range(opt.n_classes):
      result[:,i:i+1] = validity[:,i:i+1] + validity[:,i+10:i+11]

    res = torch.argmax(result, axis = 1)

    for j in range(batch_size):
      total += 1
      a = labels[j].cpu()
      if res[j] == a:
        acc += 1

print('accuracy: ', acc/total*100)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


accuracy:  85.82513829133374


## Classifier Trained with Soft Label

In [153]:
soft_model = Discriminator(opt.latent_dim)
# soft_model = discriminator
soft_model.cuda()
optimizer_soft = torch.optim.SGD(soft_model.parameters(), lr=0.1)

In [154]:
soft_model.train()

Discriminator(
  (conv1_1): Conv2d(3, 50, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv1_2): Conv2d(10, 50, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(100, 200, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2_bn): BatchNorm2d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(200, 400, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv3_bn): BatchNorm2d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(400, 20, kernel_size=(4, 4), stride=(1, 1))
)

### Train with soft label

In [155]:
for epoch in range(opt.n_epochs):
    batch_size = 512
    # Adversarial ground truths
    optimizer_soft.zero_grad()

    # Sample noise and labels as generator input
    noise = torch.randn((batch_size, 100)).view(-1, 100, 1, 1)
    fake_labels = (torch.rand(batch_size, 1) * 10).type(torch.LongTensor).squeeze()
    gen_fake_labels = onehot[fake_labels]   
    dis_fake_labels = fill[fake_labels]
    noise, gen_fake_labels, dis_fake_labels = Variable(noise.cuda()), Variable(gen_fake_labels.cuda()), Variable(dis_fake_labels.cuda())
    gen_imgs = generator(noise, gen_fake_labels)

    fill_labels = (torch.rand(batch_size, 1) * 10).type(torch.LongTensor).squeeze()
    
    fill_labels = fill[fill_labels]
    fill_labels.zero_()
    fill_labels = fill_labels.cuda()

    # get sfot label from original model
    validity = original_model(gen_imgs) # 64，20
    validity = validity.squeeze()

    # print(validity[0])
    # print(gen_labels[0])
    # print(fake_labels[0])
    # print(dasdas)
    # validity = torch.argmax(validity, axis=1).reshape((64,1))
    result = validity

    result = Variable(FloatTensor(result))
    # for i in range(opt.n_classes):
    #   result[:,i:i+1] = validity[:,i:i+1] + validity[:,i+10:i+11] # soft_model [64,10]

    # train model 2
    validity3, _ = soft_model(gen_imgs, fill_labels)
    validity3 = validity3.squeeze()

    result3 = np.zeros((batch_size, opt.n_classes))

    result3 = Variable(FloatTensor(result3))
    for i in range(opt.n_classes):
      result3[:,i:i+1] = validity3[:,i:i+1] + validity3[:,i+10:i+11] # soft_model [64,10]

    target = gen_fake_labels.view(-1,10)

    loss_soft = classification_loss(result3, fake_labels.cuda())

    loss_soft.backward()

    optimizer_soft.step()

    print("[epoch: %d], [loss: %f]"% (epoch, loss_soft.item()))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1614: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[epoch: 0], [loss: 2.306061]
[epoch: 1], [loss: 2.293617]
[epoch: 2], [loss: 2.256822]
[epoch: 3], [loss: 2.210086]
[epoch: 4], [loss: 2.231856]
[epoch: 5], [loss: 2.203026]
[epoch: 6], [loss: 2.137093]
[epoch: 7], [loss: 2.132784]
[epoch: 8], [loss: 2.081887]
[epoch: 9], [loss: 2.043584]
[epoch: 10], [loss: 2.028925]
[epoch: 11], [loss: 2.006442]
[epoch: 12], [loss: 1.970642]
[epoch: 13], [loss: 1.989500]
[epoch: 14], [loss: 1.931646]
[epoch: 15], [loss: 1.905941]
[epoch: 16], [loss: 1.902651]
[epoch: 17], [loss: 1.856733]
[epoch: 18], [loss: 1.828402]
[epoch: 19], [loss: 1.815016]
[epoch: 20], [loss: 1.780005]
[epoch: 21], [loss: 1.754904]
[epoch: 22], [loss: 1.751803]
[epoch: 23], [loss: 1.724553]
[epoch: 24], [loss: 1.723915]
[epoch: 25], [loss: 1.766324]
[epoch: 26], [loss: 1.677279]
[epoch: 27], [loss: 1.647995]
[epoch: 28], [loss: 1.637587]
[epoch: 29], [loss: 1.604537]
[epoch: 30], [loss: 1.577108]
[epoch: 31], [loss: 1.594735]
[epoch: 32], [loss: 1.593780]
[epoch: 33], [loss: 

### Train with original data

In [156]:
for epoch in range(opt.n_epochs):
    for j, (imgs, labels) in enumerate(dataloader):
        batch_size = imgs.shape[0]
        # Adversarial ground truths
        optimizer_soft.zero_grad()

        fill_labels = fill[labels]
        fill_labels.zero_()
        fill_labels = fill_labels.cuda()
        imgs = Variable(imgs.cuda())
        validity3, _ = soft_model(imgs, fill_labels) # 64，20
        validity3 = validity3.squeeze()

        # validity = torch.argmax(validity, axis=1).reshape((64,1))
        result3 = np.zeros((batch_size, opt.n_classes))

        result3 = Variable(FloatTensor(result3))
        for i in range(opt.n_classes):
          result3[:,i:i+1] = validity3[:,i:i+1] + validity3[:,i+10:i+11]
        target = onehot[labels].view(-1,10)

        loss_soft = classification_loss(result3, labels.cuda())

        loss_soft.backward()

        optimizer_soft.step()

    print("[epoch: %d], [loss: %f]"% (epoch, loss_soft.item()))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[epoch: 0], [loss: 1.261870]
[epoch: 1], [loss: 1.109285]
[epoch: 2], [loss: 1.106809]
[epoch: 3], [loss: 1.173855]
[epoch: 4], [loss: 1.165029]
[epoch: 5], [loss: 0.982257]
[epoch: 6], [loss: 0.992463]
[epoch: 7], [loss: 1.102324]
[epoch: 8], [loss: 0.874751]
[epoch: 9], [loss: 0.936988]
[epoch: 10], [loss: 0.940841]
[epoch: 11], [loss: 0.897498]
[epoch: 12], [loss: 1.060637]
[epoch: 13], [loss: 0.843511]
[epoch: 14], [loss: 1.001341]
[epoch: 15], [loss: 0.878635]
[epoch: 16], [loss: 0.929477]
[epoch: 17], [loss: 0.873344]
[epoch: 18], [loss: 0.839230]
[epoch: 19], [loss: 0.815589]
[epoch: 20], [loss: 0.858206]
[epoch: 21], [loss: 0.886705]
[epoch: 22], [loss: 0.814701]
[epoch: 23], [loss: 0.864573]
[epoch: 24], [loss: 0.828311]
[epoch: 25], [loss: 0.861515]
[epoch: 26], [loss: 0.860418]
[epoch: 27], [loss: 0.906210]
[epoch: 28], [loss: 0.842168]
[epoch: 29], [loss: 0.816369]
[epoch: 30], [loss: 0.821583]
[epoch: 31], [loss: 0.826721]
[epoch: 32], [loss: 0.850408]
[epoch: 33], [loss: 

In [158]:
total = 0
acc = 0
soft_model.eval()
for e, (imgs, labels) in enumerate(testloader):
    batch_size = imgs.shape[0]

    # Configure input
    real_imgs = Variable(imgs.type(FloatTensor))
    labels = Variable(labels.type(LongTensor))

    fill_labels = (torch.rand(batch_size, 1) * 10).type(torch.LongTensor).squeeze()
    
    fill_labels = fill[fill_labels]
    fill_labels.zero_()
    fill_labels = fill_labels.cuda()
    validity, _ = soft_model(real_imgs, fill_labels) # 64，20
    validity = validity.squeeze()
    result = torch.zeros((batch_size, opt.n_classes))

    for i in range(opt.n_classes):
      result[:,i:i+1] = validity[:,i:i+1] + validity[:,i+10:i+11]

    res = torch.argmax(result, axis = 1)

    for j in range(batch_size):
      total += 1
      a = labels[j].cpu()
      if res[j] == a:
        acc += 1

print('accuracy: ', acc/total*100)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


accuracy:  85.82897971727105
